In this notebook, we will finetune the transformer model 'ALBERT' on the AIS dataset and keep track of the performance of the model for a number of 10 to 15 epochs in total. We will also examine the performance of different interpretability techiniques on ALBERT. Lime was not included in the code.

In [ ]:
#We first need to conect to our drive, in order to access the projects files and store results
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Thesis')

In [ ]:
#Now, it is time to install the appropriate version of the transformers library
!pip install transformers-interpret==0.5.2
!pip install transformers==4.15.0
!pip install lime==0.2.0.1 #this line is included in order for 'myExplainers.py' to load properly
!pip install transformers[torch]

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00
  Created wheel for transformers-interpret: filename=transformers_interpret-0.5.2-py3-none-any.whl size=23501 sha256=6a77fa1f52514b239e0d641b3dd4b30d78d112de9e071f557ebefcd6defd6995
  Stored in directory: /root/.cache/pip/wheels/3f/b6/91/4103e1d2856cf07bb4840a08156f08edf2f63408132ea02cfd
Successfully built transformers-interpret
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 62.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#Imports of libraries required for finetuning and explaining ALBERT
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, average_precision_score
from sklearn.model_selection import train_test_split
from helper import print_results, print_results_ap
from sklearn.preprocessing import maxabs_scale
from myModel import MyModel, MyDataset
from myEvaluation import MyEvaluation
from myExplainers import MyExplainer
from scipy.special import softmax
from dataset import Dataset
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import numpy as np
import warnings
import datetime
import pickle
import torch
import time
import csv
import re

In [ ]:
#defining the paths of the model and data
data_path = '/content/drive/MyDrive/Thesis/'
model_path = '/content/drive/MyDrive/Thesis/'
save_path = '/content/drive/MyDrive/Thesis/Results/'

Now, it is time to name the model and to define the parameters of 'MyModel' class that loads transformer models.

In [ ]:
model_name = 'albert'
existing_rationales = False #no explanations
task = 'single_label' #single-label
sentence_level = False #token level
labels = 2 #two labels

Now, let us load the AIS dataset, through the 'dataset.py' file and the 'load_AIS' function. X: are the instances, y: are the labels. The 'Dataset' class of 'dataset.py' is utilized.

In [ ]:
ais = Dataset(path = data_path) #Dataset class is in 'dataset.py': parameters (path, x=None, y=None, rationales=None ,label_names=None)
x, y, label_names = ais.load_AIS() #function in Dataset class to load AIS dataset
label_names = ['class a', 'class b'] #the names for each of the two labels

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(x, y, test_size=.2, random_state=42)

size = (0.1 * len(y)) / len(train_labels)
train_texts, validation_texts, train_labels, validation_labels = train_test_split(list(train_texts), train_labels, test_size=size, random_state=42)

Now the dataset is not in the appropriate form for the transformer to process. It is necessary to define the tokenizer of the model, so as to call 'myDataset' class in 'myModel.py'.

In [ ]:
from transformers import AlbertTokenizerFast

#unlike BERT and Distilbert, ALBERT does not contain 'cs'
tokenizer = AlbertTokenizerFast.from_pretrained('albert-base-v2')

Now, it is time to transform the train, test and validation sets to the appropriate form. We will use 'MyDataset' class from 'myModel.py'.

In [ ]:
train_dataset = MyDataset(train_texts, train_labels, tokenizer)
validation_dataset = MyDataset(validation_texts, validation_labels, tokenizer)
#test_dataset = MyDataset(test_texts, test_labels, tokenizer)

But before using 'MyModel' class from 'myModel.py', ALBERT should be finetuned!

In [ ]:
from transformers import Trainer, TrainingArguments
from myTransformer import AlbertForSequenceClassification as transformer_model


#calling the base pretrained RoBERTa model
model = transformer_model.from_pretrained('albert-base-v2',num_labels = len(label_names) , output_attentions=True,
                              output_hidden_states=True)

#the training arguments that we will pass to the trainer of the transformers. 15 epochs were used for training
training_arguments = TrainingArguments(evaluation_strategy='epoch', save_strategy='epoch', logging_strategy='epoch',
                                                log_level='critical', output_dir='./results', num_train_epochs=15,
                                                per_device_train_batch_size=8, per_device_eval_batch_size=8,
                                                warmup_steps=200, weight_decay=0.01, logging_dir='./logs')

#passing to the trainer the model, the arguments and all train and validation instances
trainer = Trainer(model=model, args=training_arguments, train_dataset=train_dataset, eval_dataset=validation_dataset)

#Let's train the model!
trainer.train()

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model 

Epoch,Training Loss,Validation Loss
1,0.289400,0.551484
2,0.187900,0.429795
3,0.317700,0.258993
4,0.328100,0.160868
5,0.145700,0.165334
6,0.201500,0.396848
7,0.242700,0.113609
8,0.133300,0.090235
9,0.094300,0.100048
10,0.077600,0.103589


TrainOutput(global_step=3975, training_loss=0.14822510245461135, metrics={'train_runtime': 664.323, 'train_samples_per_second': 47.778, 'train_steps_per_second': 5.984, 'total_flos': 589635250869600.0, 'train_loss': 0.14822510245461135, 'epoch': 15.0})

Now, it is time to save the model in 'albert_ais' file.

In [ ]:
trainer.model.save_pretrained('/content/drive/MyDrive/Thesis/albert_ais')

Now, we can use 'MyModel' and make then make predictions.

In [ ]:
#new model
model = MyModel(model_path,'albert_ais', model_name, task, labels, 'cased')

#the maximum number of tokens a single sentence can have e.g. 512
max_sequence_len = model.tokenizer.max_len_single_sentence

#again the tokenizer is RobertaTokenizerFast, that is selected through 'MyModel' and '__load_model__' function
tokenizer = model.tokenizer

#gpu training
torch.cuda.is_available()
model.trainer.model.to('cuda')

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

Then, we measure the performance of the model using average precision score and f1 score (both macro).

In [ ]:
predictions = []

#time for predictions
starting_prediction_time = time.time()

for test_text in test_texts:
    outputs = model.my_predict(test_text)
    predictions.append(outputs[0])

#printing the total time that predictions took
ending_prediction_time = time.time()
total_time = ending_prediction_time - starting_prediction_time
print('The total time for predictions is:' ,round(total_time,3),' seconds')

The total time for predictions is: 24.223  seconds


In [ ]:
#labels of the predictions
pred_labels = []

for prediction in predictions:
    pred_labels.append(np.argmax(softmax(prediction)))

def average_precision_wrapper(y, y_pred, view):
    return average_precision_score(y, y_pred.toarray(), average=view)

#macro scores
p_s = f"Average precision score: {round(average_precision_score(test_labels, pred_labels, average='macro'),4)} %"
f1 = f"f1 score score: {round(f1_score(test_labels, pred_labels, average='macro'),4)} %"

#printing results
print(p_s)
print(f1)

Average precision score: 0.9046 %
f1 score score: 0.9691 %


We can also change the hyperparameters for training, but we notice that the performance of ALBERT is already satisfactory and the focus should be shifted on the interpretations. Let's store the results in the 'Results' file.

In [ ]:
#the data to write in the file
data = (p_s, f1)
now = datetime.datetime.now()
file_name = save_path + 'ALBERT_AIS'+str(now.day) + '_' + str(now.month) + '_' + str(now.year)

#results in files
with open(file_name+ 'PERFORMANCE_ON_AIS.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL) #data
    #pickle.dump(f1, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(file_name+'TIME_ON_AIS.pickle', 'wb') as handle:
    pickle.dump(total_time, handle, protocol=pickle.HIGHEST_PROTOCOL)

Let's ensure that the results are properly loaded from the file that we stored them.

In [ ]:
with open(file_name+'PERFORMANCE_ON_AIS.pickle', 'rb') as handle:
     performance = pickle.load(handle)
     for score in performance:
         print(score)

with open(file_name+'TIME_ON_AIS.pickle', 'rb') as handle:
     time = pickle.load(handle)
     print('The total time for predictions is:' ,round(time,3),' seconds')

Now, let us initialize the explainers and the evaluation module, as well as define the metrics that will be utilized. In this case, the following is true:
* F=Faithfulness
* FTP=RFT (Ranked Faithful Truthfulness)
* NZW=Complexity

In [ ]:
#layers are 12 this time
my_explainers = MyExplainer(label_names, model, layers=12)

#complexity, faithfulness, RFT
my_evaluators = MyEvaluation(label_names, model.my_predict, False, True, tokenizer=tokenizer) #parameters: (label_names, predict, sentence_level, evaluation_level_all=True)
my_evaluatorsP = MyEvaluation(label_names, model.my_predict, False, False, tokenizer=tokenizer)

evaluation =  {'F':my_evaluators.faithfulness, 'FTP': my_evaluators.faithful_truthfulness_penalty,
          'NZW': my_evaluators.nzw}
evaluationP = {'F':my_evaluatorsP.faithfulness, 'FTP': my_evaluatorsP.faithful_truthfulness_penalty,
          'NZW': my_evaluatorsP.nzw}

We will now measure the performance of IG.

In [ ]:
import time
with warnings.catch_warnings():

    #ignore the warnings
    warnings.simplefilter("ignore", category=RuntimeWarning)

    #date
    now = datetime.datetime.now()

    #saving results
    file_name = save_path + 'AIS_ALBERT_IG_'+str(now.day) + '_' + str(now.month) + '_' + str(now.year)

    #metrics
    metrics = {'F':[], 'FTP':[], 'NZW':[]}
    metricsP = {'F':[], 'FTP':[], 'NZW':[]}

    #time_r = [[],[]]: sublists for each technique
    time_r = [ [] ] #now only ig is present

    #neighnbors
    #my_explainers.neighbours = 2000

    #ig
    techniques = [my_explainers.ig]

    #for each test instance
    for ind in tqdm(range(len(test_texts))): #progress bar

        #to not run out of memory
        torch.cuda.empty_cache()

        #the instance of test set
        instance = test_texts[ind]

        #reseting the state memory
        my_evaluators.clear_states()
        my_evaluatorsP.clear_states()

        #prediction, attention matrix and hidden states. Here we care about predictions
        prediction, _, _ = model.my_predict(instance)

        #RobetaTokenizerFast
        enc = model.tokenizer([instance,instance], truncation=True, padding=True)[0] #first element of output dict: input IDs

        #real tokens or padding: extracting the mask
        mask = enc.attention_mask

        #extract special tokens
        tokens = enc.tokens

        interpretations = []
        kk = 0

        #ig now. This piece of code did not change. because other techniques will be included later
        for technique in techniques:
            ts = time.time()

            #returns interpretations
            temp = technique(instance, prediction, tokens, enc.ids, _, _) #no attention and hidden states

            #normalization in interpretations
            interpretations.append([np.array(i)/np.max(abs(np.array(i))) for i in temp])

            #append the time it took
            time_r[kk].append(time.time()-ts)
            kk = kk + 1

        #'F','FTP','NZW'
        for metric in metrics.keys():
            evaluated = []
            for interpretation in interpretations:

                #all parameters: interpretation, tweaked_interpretation, instance, prediction, tokens, hidden_states, t_hidden_states, rationales
                evaluated.append(evaluation[metric](interpretation, _, instance, prediction, tokens, _, _, _))

            #save evaluations in dict
            metrics[metric].append(evaluated)

        #copy of saved state
        my_evaluatorsP.saved_state = my_evaluators.saved_state.copy()

        #clear again all states
        my_evaluators.clear_states()

        for metric in metrics.keys():
            evaluatedP = []
            for interpretation in interpretations:

                #in a similar way as 'evaluation'
                evaluatedP.append(evaluationP[metric](interpretation, _, instance, prediction, tokens, _, _, _))

            #save evaluations
            metricsP[metric].append(evaluatedP)

        #write results to files
        with open(file_name+'(A).pickle', 'wb') as handle:
            pickle.dump(metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(file_name+'(P).pickle', 'wb') as handle:
            pickle.dump(metricsP, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(file_name+'_TIME.pickle', 'wb') as handle:
            pickle.dump(time_r, handle, protocol=pickle.HIGHEST_PROTOCOL)

time_r = np.array(time_r)
time_r.mean(axis=1)

100%|██████████| 605/605 [51:07<00:00,  5.07s/it]


array([0.544615])

In [ ]:
print(time_r)
print(time_r.mean(axis=1))

[[0.49058962 0.4830153  0.48891187 0.46024466 0.49483275 0.63833618
  0.86571121 0.53239799 0.52989101 0.5050447  0.50394416 0.75027442
  0.7047348  0.58048558 0.44723463 0.46435285 0.45258021 0.47201681
  0.47891521 0.85614133 0.63290071 0.54309797 0.47325945 0.79707718
  0.70338845 0.47446632 0.48154116 0.47387123 0.47791862 0.55173755
  0.46652484 0.4300189  0.43684506 0.47549653 0.64097905 0.44040656
  1.68172956 0.4244523  0.56052136 0.48964977 0.47551394 0.45048046
  0.6150105  0.44523048 0.67693019 0.46519613 0.41277242 0.48684049
  0.43843532 0.50258422 0.59435201 0.5139482  0.42998791 0.45058012
  0.47347617 0.46109438 0.48057914 0.47239923 0.46644926 0.43111229
  0.50995207 0.48303723 0.64477992 1.08524585 0.47910571 0.50333691
  0.99528813 0.6141572  0.48134875 0.54400134 0.45677447 0.63910842
  0.48300266 0.46431375 0.63291383 0.44342136 0.43013692 1.09514832
  0.60738254 0.48596144 0.88323832 0.45873165 0.46612549 0.47517514
  0.4457655  0.44868493 0.44269228 0.43598318 0.

Now, let us print the results for IG.

In [ ]:
print_results(file_name+'(A)', [' IG  '], metrics, label_names)

F
 IG    0.05974999815225601 | 0.00771 0.11179
FTP
 IG    0.17886 | 0.17886 0.17886
NZW
 IG    1.0 | 1.0 1.0


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
print_results(file_name+'(P)', [' IG  '], metricsP, label_names)

F
 IG    0.41282 | 0.01679 0.80884
FTP
 IG    0.54929 | 0.03645 1.06212
NZW
 IG    1.0 | 1.0 1.0


We will now experiment on various attention setups.

In [ ]:
conf = []
#'Mean', 'Multi', 0, 1, 2, 3, 4, 5
for ci in ['Mean', 'Multi'] + list(range(12)):

    #'Mean', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11
    for ce in ['Mean'] + list(range(12)):

        # Matrix: From, To, MeanColumns, MeanRows, MaxColumns, MaxRows (rows?)
        for cp in ['From', 'To', 'MeanColumns', 'MaxColumns']:

            # Selection: True: select layers per head, False: do not
            for cl in [False]:
                conf.append([ci, ce, cp, cl])

len(conf) #8*13*4*1

728

In [ ]:
import time
with warnings.catch_warnings():

    #ignore the warnings
    warnings.simplefilter("ignore", category=RuntimeWarning)

    #date
    now = datetime.datetime.now()

    #saving results
    file_name = save_path + 'AIS_ALBERT_ATTENTION_'+str(now.day) + '_' + str(now.month) + '_' + str(now.year)

    #metrics
    metrics = {'FTP':[], 'F':[], 'NZW':[]}
    metricsP = {'FTP':[], 'F':[], 'NZW':[]}

    #times
    time_r = []
    time_b = []
    time_b2 = []

    #attentions setups
    for con in conf:
        time_r.append([])

    for ind in tqdm(range(len(test_texts))):

        #to not run out of memory
        torch.cuda.empty_cache()

        #one instance
        instance = test_texts[ind]

        #clear states of evaluators
        my_evaluators.clear_states()
        my_evaluatorsP.clear_states()

        #save calculated configurations
        my_explainers.save_states = {}

        #prediction, attention matrix and hidden states. Here we care about predictions and attention.
        prediction, attention, _ = model.my_predict(instance)

        #RobertaTokenizerFast
        enc = model.tokenizer([instance,instance], truncation=True, padding=True)[0]

        #real tokens or padding: extracting the mask
        mask = enc.attention_mask

        #extract special tokens
        tokens = enc.tokens

        interpretations = []
        kk = 0
        for con in conf:

            #time
            ts = time.time()

            #set configuration
            my_explainers.config = con

            #returns interpretations
            temp = my_explainers.my_attention(instance, prediction, tokens, mask, attention, _) #no hidden states

            #scaling interpretations
            interpretations.append([maxabs_scale(i) for i in temp])

            #append time
            time_r[kk].append(time.time()-ts)
            kk = kk + 1

        #'F','FTP','NZW'
        for metric in metrics.keys():
            evaluated = []
            k = 0

            for interpretation in interpretations:
                tt = time.time()

                #all parameters: interpretation, tweaked_interpretation, instance, prediction, tokens, hidden_states, t_hidden_states, rationales
                evaluated.append(evaluation[metric](interpretation, _, instance, prediction, tokens, _, _, _))
                k = k + (time.time()-tt) #time
            if metric == 'FTP':
                time_b.append(k)
            metrics[metric].append(evaluated)

        my_evaluatorsP.saved_state = my_evaluators.saved_state.copy()

        for metricP in metricsP.keys():
            evaluated = []
            k = 0

            for interpretation in interpretations:
                tt = time.time()

                #all parameters: interpretation, tweaked_interpretation, instance, prediction, tokens, hidden_states, t_hidden_states, rationales
                evaluated.append(evaluationP[metricP](interpretation, _, instance, prediction, tokens, _, _, _))
                k = k + (time.time()-tt)

            if metricP == 'FTP':
                time_b2.append(k)
            metricsP[metricP].append(evaluated)

        if(ind != 0):
            with open(file_name+' (A).pickle', 'rb') as handle:
                old_metrics = pickle.load(handle)
            with open(file_name+' (P).pickle', 'rb') as handle:
                old_metricsP = pickle.load(handle)

            #append new results
            for key in metrics.keys():
                old_metrics[key].append(metrics[key][0])
                old_metricsP[key].append(metricsP[key][0])
        else:
            old_metrics = metrics
            old_metricsP = metricsP

        #save metrics as below
        with open(file_name+' (A).pickle', 'wb') as handle:
            pickle.dump(old_metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(file_name+' (P).pickle', 'wb') as handle:
            pickle.dump(old_metricsP, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(file_name+'_TIME.pickle', 'wb') as handle:
            pickle.dump(time_r, handle, protocol=pickle.HIGHEST_PROTOCOL)

        del old_metrics,old_metricsP
        metrics = {'FTP':[], 'F':[], 'NZW':[]}
        metricsP = {'FTP':[], 'F':[], 'NZW':[]}

#times
time_r = np.array(time_r)
time_r.mean(axis=1).min(),time_r.mean(axis=1).max(), time_r.mean(axis=1).mean(), time_r.sum(axis=1).mean(), np.mean(time_b), np.mean(time_b2)

100%|██████████| 605/605 [5:53:44<00:00, 35.08s/it]


(0.0015491718103077786,
 0.004991729200378922,
 0.001938360068791786,
 1.1727078416190304,
 17.51778496395458,
 6.637546705608526)

In [ ]:
print(time_r)
print(time_r.mean(axis=1).min())
time_r.mean(axis=1).max()
time_r.sum(axis=1).mean()
print(time_b)
np.mean(time_b)
print(time_b2)
np.mean(time_b2)

[[0.00165343 0.00130343 0.00132394 ... 0.00421238 0.00152063 0.00133038]
 [0.00105715 0.0008955  0.00088096 ... 0.00214458 0.00101328 0.00088024]
 [0.00103021 0.00088096 0.00082421 ... 0.00216794 0.00097728 0.00083232]
 ...
 [0.00086451 0.00063443 0.00066161 ... 0.00223088 0.00081587 0.00067854]
 [0.00084352 0.00070095 0.00067425 ... 0.00217652 0.00081706 0.00069332]
 [0.00075459 0.00066566 0.00067258 ... 0.00175023 0.00081968 0.00063992]]
0.0015491718103077786
[5.316105604171753, 3.175027370452881, 3.1837122440338135, 6.553654193878174, 9.11928915977478, 29.96585774421692, 68.35091710090637, 14.714859008789062, 13.868924379348755, 14.412672519683838, 12.241395711898804, 43.233644008636475, 38.23515439033508, 23.80863928794861, 6.127811908721924, 3.238935708999634, 4.162184953689575, 7.431138515472412, 7.253055810928345, 62.05953049659729, 25.646110773086548, 15.193243980407715, 5.804856061935425, 55.682724475860596, 38.07408308982849, 3.819981813430786, 2.9793550968170166, 2.745823860

6.637546705608526

In [ ]:
#print_results(file_name+' (A)', conf, metrics, label_names)

with open(file_name+' (A).pickle', 'rb') as handle:
    metrics = pickle.load(handle)

In [ ]:
#print_results(file_name+' (P)', conf, metricsP, label_names)

with open(file_name+' (P).pickle', 'rb') as handle:
    metricsP = pickle.load(handle)

We calculate the best attention setup using Optimus variations (we do not use the Optimus implementation at this step).

In [ ]:
print_results_ap(metrics, label_names, conf)

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Baseline: 1.0674608780236738e-08  and NZW: 1.0
Max Across: 1.7679416840221673e-08  and NZW: 1.0
Per Label Per Instance: 0.12319040475393231  and NZW:  0.9999786263892847
Per Instance: 6.283432414655454e-08  and NZW:  0.9998830606622815


In [ ]:
print_results_ap(metricsP, label_names, conf)

Baseline: 0.4851533115246219  and NZW: 1.0
Max Across: 0.6821898596097431  and NZW: 1.0
Per Label Per Instance: 0.7940377324379158  and NZW:  0.9998460591133005
Per Instance: 0.7940377324379158  and NZW:  0.9998460591133005


We repeat the process with Attention Scores with negative values (A*), thus by skipping the Softmax function. In the attention setups, we exclude the multiplication option in heads and layers, as a few combinations reach +/-inf.

In [ ]:
conf = []
for ci in ['Mean'] + list(range(12)):
    for ce in ['Mean'] + list(range(12)):
        for cp in ['From', 'To', 'MeanColumns', 'MaxColumns']: # Matrix: From, To, MeanColumns, MeanRows, MaxColumns, MaxRows
            for cl in [False]: # Selection: True: select layers per head, False: do not
                conf.append([ci, ce, cp, cl])
len(conf)

676

In [ ]:
import time
import math
with warnings.catch_warnings():

    warnings.simplefilter("ignore", category=RuntimeWarning)

    now = datetime.datetime.now()

    file_name = save_path + 'AIS_ALBERT_A_ATTENTION_NO_SOFTMAX_'+str(now.day) + '_' + str(now.month) + '_' + str(now.year)

    metrics = {'FTP':[], 'F':[], 'NZW':[]}
    metricsP = {'FTP':[], 'F':[], 'NZW':[]}

    time_r = []
    time_b = []
    time_b2 = []

    for con in conf:
        time_r.append([])

    for ind in tqdm(range(len(test_texts))):

        torch.cuda.empty_cache()

        instance = test_texts[ind]

        my_evaluators.clear_states()
        my_evaluatorsP.clear_states()

        my_explainers.save_states = {}

        prediction, _, hidden_states = model.my_predict(instance)

        enc = model.tokenizer([instance,instance], truncation=True, padding=True)[0]

        mask = enc.attention_mask

        tokens = enc.tokens

        attention = []

        for la in range(12):
            our_new_layer = []
            bob = model.trainer.model.albert.encoder.albert_layer_groups[0].albert_layers[0].attention
            has = hidden_states[la]

            aaa = bob.key(torch.tensor(has).to('cuda'))
            bbb = bob.query(torch.tensor(has).to('cuda'))
            for he in range(12):
                attention_scores = torch.matmul(bbb[:,he*64:(he+1)*64], aaa[:,he*64:(he+1)*64].transpose(-1, -2))
                attention_scores = attention_scores / math.sqrt(64)
                our_new_layer.append(attention_scores.cpu().detach().numpy())
            attention.append(our_new_layer)
        attention = np.array(attention)

        interpretations = []
        kk = 0
        for con in conf:
            ts = time.time()
            my_explainers.config = con
            temp = my_explainers.my_attention(instance, prediction, tokens, mask, attention, _)
            interpretations.append([maxabs_scale(i) for i in temp])
            time_r[kk].append(time.time()-ts)
            kk = kk + 1

        #print(interpretations[0])
        for metric in metrics.keys():
            evaluated = []
            k = 0
            for interpretation in interpretations:
                tt = time.time()
                evaluated.append(evaluation[metric](interpretation, _, instance, prediction, tokens, _, _, _))
                k = k + (time.time()-tt)
            if metric == 'FTP':
                time_b.append(k)
            metrics[metric].append(evaluated)
        my_evaluatorsP.saved_state = my_evaluators.saved_state.copy()
        for metric in metrics.keys():
            evaluated = []
            k = 0
            for interpretation in interpretations:
                tt = time.time()
                evaluated.append(evaluationP[metric](interpretation, _, instance, prediction, tokens, _, _, _))
                k = k + (time.time()-tt)
            if metric == 'FTP':
                time_b2.append(k)
            metricsP[metric].append(evaluated)
        with open(file_name+' (A).pickle', 'wb') as handle:
            pickle.dump(metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(file_name+' (P).pickle', 'wb') as handle:
            pickle.dump(metricsP, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(file_name+'_TIME.pickle', 'wb') as handle:
            pickle.dump(time_r, handle, protocol=pickle.HIGHEST_PROTOCOL)
time_r = np.array(time_r)
time_r.mean(axis=1).min(),time_r.mean(axis=1).max(), time_r.mean(axis=1).mean(), time_r.sum(axis=1).mean(), np.mean(time_b), np.mean(time_b2)

In [ ]:
print_results(file_name+' (A)', conf, metrics, label_names)

In [ ]:
print_results(file_name+' (P)', conf, metricsP, label_names)

We calculate the best attention setup using Optimus variations.

In [ ]:
print_results_ap(metrics, label_names, conf)

In [ ]:
print_results_ap(metricsP, label_names, conf)